# Triplet Loss with Faces -- Fairness Study

## Initialization

In [ ]:
# !pip install torch

In [ ]:
# !pip install mat73

In [ ]:
# !pip install wandb -qqq
# import wandb

In [ ]:
# !git clone https://github.com/mgornet/CNPEN

### Check device

In [1]:
!nvidia-smi

Tue Jun 21 10:00:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A400...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   57C    P8    13W /  N/A |   3184MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Import Librairies

In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from time import perf_counter
from typing import Callable
import itertools
import mat73
import pandas as pd
import re

import sys
import os
import tarfile

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

import torch
from torch import nn, optim
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

import os.path as op
try:
    from urllib.request import urlretrieve
except ImportError:  # Python 2 compat
    from urllib import urlretrieve
    
from sklearn.metrics import confusion_matrix, auc, roc_curve, \
precision_recall_curve, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from scipy import optimize

import wandb

In [3]:
%cd ./files/

from triplet import TripletGenerator, TripletLearner, TripletLoss, TripletLossRaw, \
distance, distance_vectors
from builder import create_dataframe, from_tensor_to_numpy, from_numpy_to_tensor, extend_dataframe
from prints import print_img, print_img_from_path, print_img_from_id, \
print_img_from_classid, print_from_gen, print_from_gen2, print_pair, print_hist_loss, \
print_hist_dist, print_hist_dist_zoom, print_img_category, \
print_roc, print_logistic_regression, print_prec_recall
from test_train_loops import training, testing, adaptative_train, compute_distances
from classification import authentification_img, predict, triplet_acc,\
build_df_fairness, triplet_acc_fairness, bootstrap, bootstrap_by_pairs #triplet_acc_for_bootstrap

/home/mgornet/Bureau/Code/CNPEN/files


In [4]:
!pwd

/home/mgornet/Bureau/Code/CNPEN/files


## Generate Data

### Create dataframe

In [5]:
URL = "http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz"
FILENAME = "lfw-deepfunneled.tgz"

if not op.exists(FILENAME):
    print('Downloading %s to %s...' % (URL, FILENAME))
    urlretrieve(URL, FILENAME)

if not op.exists("lfw"):
    print('Extracting image files...')
    tar = tarfile.open("lfw-deepfunneled.tgz")
    tar.extractall("lfw")
    tar.close()

In [6]:
PATH = "lfw/lfw-deepfunneled/"

In [7]:
tic = perf_counter()
df_init, all_imgs = create_dataframe()
toc = perf_counter()
print(f"DataFrame creation: {((toc - tic)/60):.1f} min")

Number of individuals:  5749
Number of total images:  13233
images weigh  0.57 GB
DataFrame creation: 0.7 min


In [8]:
tic = perf_counter()
df = extend_dataframe(df_init)
toc = perf_counter()
print(f"DataFrame extention: {((toc - tic)/60):.1f} min")

DataFrame extention: 0.1 min


### Build sets, generators and network

In [9]:
num_classes = len(df.Classid.unique())
print("Number of individuals: ", num_classes)

Number of individuals:  5749


In [10]:
indiv_min = df.Classid.min()
split_train_valid = int(num_classes * 0.75)
split_train_test = int(num_classes * 0.8)
indiv_max = df.Classid.max()

In [11]:
print(f"Train set from indiv {indiv_min} to {split_train_valid-1}")
print(f"Valid set from indiv {split_train_valid} to {split_train_test-1}")
print(f"Test set from indiv {split_train_test} to {indiv_max}")

Train set from indiv 0 to 4310
Valid set from indiv 4311 to 4598
Test set from indiv 4599 to 5748


In [12]:
df_train = df[df.Classid<split_train_valid]
df_valid = df[(df.Classid>=split_train_valid)&(df.Classid<split_train_test)]
df_test = df[df.Classid>=split_train_test]

In [13]:
print("Number of training images: ", len(df_train))
print("Number of validation images: ", len(df_valid))
print("Number of testing images: ", len(df_test))
print("Number of total images: ", len(df_train)+len(df_valid)+len(df_test))
print("len original: ", len(df))

Number of training images:  10060
Number of validation images:  586
Number of testing images:  2587
Number of total images:  13233
len original:  13233


In [14]:
print("Number of individuals in the training set: ", len(df_train.Classid.unique()))
print("Number of individuals in the validation set: ", len(df_valid.Classid.unique()))
print("Number of individuals in the testing set: ", len(df_test.Classid.unique()))

Number of individuals in the training set:  4311
Number of individuals in the validation set:  288
Number of individuals in the testing set:  1150


In [15]:
value_count = df_train.Classid.value_counts()
print("Number of individuals with more than one image in the training set: ", len(value_count[value_count.values>1]))
value_count = df_valid.Classid.value_counts()
print("Number of individuals with more than one image in the validation set: ", len(value_count[value_count.values>1]))
value_count = df_test.Classid.value_counts()
print("Number of individuals with more than one image in the testing set: ", len(value_count[value_count.values>1]))

Number of individuals with more than one image in the training set:  1267
Number of individuals with more than one image in the validation set:  79
Number of individuals with more than one image in the testing set:  334


In [16]:
df_valid.head()

,Classid,Name,Path,Male,Asian,White,Black,Baby,Child,Youth,...,Pale Skin,5 o Clock Shadow,Strong Nose-Mouth Lines,Wearing Lipstick,Flushed Face,High Cheekbones,Brown Eyes,Wearing Earrings,Wearing Necktie,Wearing Necklace
10060,4311,Pedro_Solbes,Pedro_Solbes/Pedro_Solbes_0001.jpg,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
10061,4311,Pedro_Solbes,Pedro_Solbes/Pedro_Solbes_0002.jpg,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
10062,4311,Pedro_Solbes,Pedro_Solbes/Pedro_Solbes_0003.jpg,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
10063,4311,Pedro_Solbes,Pedro_Solbes/Pedro_Solbes_0004.jpg,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10064,4312,Pedro_Velasquez,Pedro_Velasquez/Pedro_Velasquez_0001.jpg,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [17]:
BATCH_SIZE = 128 # 128
BATCH_VALID_SIZE = 128 #128 #8
BATCH_TEST_SIZE = 128 #128 #32

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
margin = 0.2
criterion = TripletLoss(margin)
criterion_test = TripletLossRaw(margin)

### Load Model

In [19]:
# Load pretrained model

model = TripletLearner(base_channels=32)
model.load_state_dict(torch.load("../models/in_article/base_model.pth",map_location=torch.device('cpu')))
model = model.to(device)
model.eval()

TripletLearner(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16): ReLU()
    (17): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), 

### Determine THRESHOLD

In [20]:
# Build THRESHOLD (for details, see the notebook "determine_threshold")

gen = TripletGenerator(df_valid, all_imgs, BATCH_VALID_SIZE, device, model, margin)
loader = DataLoader(gen, batch_size=None, shuffle=True)

list_loader = []
for _ in range(10):
    list_loader.extend(list(loader))

pos_dist, neg_dist, _ = compute_distances(list_loader, device, model) #loader

y_pos = [1 for _ in range(len(pos_dist))]
y_neg = [0 for _ in range(len(neg_dist))]

y = y_pos + y_neg
X = pos_dist + neg_dist
Xmoins = np.array(X)*(-1)
Xlogistic = np.array(Xmoins).reshape(-1,1)

clf = LogisticRegression(random_state=0).fit(Xlogistic, y)

THRESHOLD = (clf.intercept_/clf.coef_)[0,0]
print("THRESHOLD with logistic regression:", THRESHOLD)

/home/mgornet/Bureau/Code/CNPEN/files/triplet.py:224: UserWarning: Batch size number was changed from 128 to 79 because there are only 79 individuals with more than 2 pictures.
  warnings.warn(


Processing:   0%|          | 0/10 [00:00<?, ?it/s]

THRESHOLD with logistic regression: 0.7300254046243199


## Fairness study

### Build data fairness

In [21]:
gen = TripletGenerator(df_test, all_imgs, BATCH_TEST_SIZE, device, model, margin, return_id=True)

/home/mgornet/Bureau/Code/CNPEN/files/triplet.py:234: UserWarning: Number of unique identities with more than 2 pictures (334) is not divisible by batch_size (128). Remainder: 78
  warnings.warn(


In [ ]:
# tic = perf_counter()
# df_fairness = build_df_fairness(all_imgs, df_test, gen, 20, device, model, THRESHOLD)
# toc = perf_counter()
# print(f"DataFrame creation: {((toc - tic)/60):.1f} min")

In [ ]:
tic = perf_counter()
df_fairness = build_df_fairness(all_imgs, df_test.append(df_valid), gen, 50, device, model, THRESHOLD)
toc = perf_counter()
print(f"DataFrame creation: {((toc - tic)/60):.1f} min")

In [ ]:
df_fairness

In [ ]:
df_fairness.describe()

### Look at general stats

In [ ]:
print("Same identity - mean distance: ", df_fairness[df_fairness.y_true==1].Distance.mean())
print("Same identity - std distance: ", df_fairness[df_fairness.y_true==1].Distance.std())
print("Same identity - percentiles: ", np.percentile(df_fairness[df_fairness.y_true==1]['Distance'], [5,25,50,75,95]))
print("\n")
print("Different identities - mean distance: ", df_fairness[df_fairness.y_true==0].Distance.mean())
print("Different identities - std distance: ", df_fairness[df_fairness.y_true==0].Distance.std())
print("Different identities - percentiles: ", np.percentile(df_fairness[df_fairness.y_true==0]['Distance'], [5,25,50,75,95]))
print("\n")
print("Mean accuracy: ", df_fairness['correct_predict'].mean())
print("Bootstrapping mean accuracy: ", bootstrap(df_fairness, agg_func=lambda df: df['correct_predict'].mean()))
print("\n")
print("Triplet accuracy: ", triplet_acc_fairness(df_fairness))

In [ ]:
pos_dist = df_fairness[df_fairness.y_true==1]['Distance']
neg_dist = df_fairness[df_fairness.y_true==0]['Distance']
print_hist_dist_zoom(pos_dist, neg_dist, zoom=5.)

In [ ]:
X = -np.array(df_fairness.Distance)
y = np.array(df_fairness.y_true)
y_pred = np.array(df_fairness.y_pred)

In [ ]:
fpr_dist, tpr_dist, thresholds_dist = roc_curve(y, X)
roc_auc_dist = auc(fpr_dist,tpr_dist)

precision_dist, recall_dist, thresholds_recall_dist = precision_recall_curve(y, X)
auc_s_dist = auc(recall_dist, precision_dist)

In [ ]:
print_roc(fpr_dist, tpr_dist, roc_auc_dist)
print_prec_recall(precision_dist, recall_dist, auc_s_dist)

In [ ]:
tn,fp,fn,tp = confusion_matrix(y, y_pred).ravel()

TPR = tp/(tp+fp)
FPR = fp/(tp+fp)
TNR = tn/(tn+fn)
FNR = fn/(tn+fn)

In [ ]:
print("Confusion Matrix Total")
print(confusion_matrix(y, y_pred))

print("\n","Accuracy score:",accuracy_score(y, y_pred))
 
print("\n", "f1 score:", f1_score(y, y_pred), "\n")

print('TPR: ', TPR)
print('FPR: ', FPR)
print('TNR: ', TNR)
print('FNR: ', FNR)

### Stats in subgroups

#### White Male vs Non White Male

In [ ]:
pos_dist = df_fairness[(df_fairness.y_true==1) & (df_fairness.AB_WhiteMale==1)]['Distance']
neg_dist = df_fairness[(df_fairness.y_true==0) & (df_fairness.AB_WhiteMale==1)]['Distance']
print_hist_dist_zoom(pos_dist, neg_dist, zoom=5.)

In [ ]:
pos_dist = df_fairness[(df_fairness.y_true==1) & (df_fairness.AB_NoWhiteMale==1)]['Distance']
neg_dist = df_fairness[(df_fairness.y_true==0) & (df_fairness.AB_NoWhiteMale==1)]['Distance']
print_hist_dist_zoom(pos_dist, neg_dist, zoom=5.)

In [ ]:
pos_dist = df_fairness[(df_fairness.y_true==1) & (df_fairness.AB_WhiteMale==0)]['Distance']
neg_dist = df_fairness[(df_fairness.y_true==0) & (df_fairness.AB_WhiteMale==0)]['Distance']
print_hist_dist_zoom(pos_dist, neg_dist, zoom=5.)

In [ ]:
y_AB_WhiteMale = np.array(df_fairness[df_fairness['AB_WhiteMale']==1].y_true)
y_pred_AB_WhiteMale = np.array(df_fairness[df_fairness['AB_WhiteMale']==1].y_pred)

tn_AB_WhiteMale,fp_AB_WhiteMale,fn_AB_WhiteMale,tp_AB_WhiteMale = confusion_matrix(y_AB_WhiteMale, y_pred_AB_WhiteMale).ravel()

TPR_AB_WhiteMale = tp_AB_WhiteMale/(tp_AB_WhiteMale+fp_AB_WhiteMale)
FPR_AB_WhiteMale = fp_AB_WhiteMale/(tp_AB_WhiteMale+fp_AB_WhiteMale)
TNR_AB_WhiteMale = tn_AB_WhiteMale/(tn_AB_WhiteMale+fn_AB_WhiteMale)
FNR_AB_WhiteMale = fn_AB_WhiteMale/(tn_AB_WhiteMale+fn_AB_WhiteMale)

y_AB_NoWhiteMale = np.array(df_fairness[df_fairness['AB_NoWhiteMale']==1].y_true)
y_pred_AB_NoWhiteMale = np.array(df_fairness[df_fairness['AB_NoWhiteMale']==1].y_pred)

tn_AB_NoWhiteMale,fp_AB_NoWhiteMale,fn_AB_NoWhiteMale,tp_AB_NoWhiteMale = confusion_matrix(y_AB_NoWhiteMale, y_pred_AB_NoWhiteMale).ravel()

TPR_AB_NoWhiteMale = tp_AB_NoWhiteMale/(tp_AB_NoWhiteMale+fp_AB_NoWhiteMale)
FPR_AB_NoWhiteMale = fp_AB_NoWhiteMale/(tp_AB_NoWhiteMale+fp_AB_NoWhiteMale)
TNR_AB_NoWhiteMale = tn_AB_NoWhiteMale/(tn_AB_NoWhiteMale+fn_AB_NoWhiteMale)
FNR_AB_NoWhiteMale = fn_AB_NoWhiteMale/(tn_AB_NoWhiteMale+fn_AB_NoWhiteMale)

y_AoB_NoWhiteMale = np.array(df_fairness[df_fairness['AB_WhiteMale']==0].y_true)
y_pred_AoB_NoWhiteMale = np.array(df_fairness[df_fairness['AB_WhiteMale']==0].y_pred)

tn_AoB_NoWhiteMale,fp_AoB_NoWhiteMale,fn_AoB_NoWhiteMale,tp_AoB_NoWhiteMale = confusion_matrix(y_AoB_NoWhiteMale, y_pred_AoB_NoWhiteMale).ravel()

TPR_AoB_NoWhiteMale = tp_AoB_NoWhiteMale/(tp_AoB_NoWhiteMale+fp_AoB_NoWhiteMale)
FPR_AoB_NoWhiteMale = fp_AoB_NoWhiteMale/(tp_AoB_NoWhiteMale+fp_AoB_NoWhiteMale)
TNR_AoB_NoWhiteMale = tn_AoB_NoWhiteMale/(tn_AoB_NoWhiteMale+fn_AoB_NoWhiteMale)
FNR_AoB_NoWhiteMale = fn_AoB_NoWhiteMale/(tn_AoB_NoWhiteMale+fn_AoB_NoWhiteMale)

In [ ]:
print("A and B White Male - mean accuracy: ", df_fairness[df_fairness['AB_WhiteMale']==1]['correct_predict'].mean())
print("A and B White Male - bootstrapping mean accuracy: ", bootstrap(df_fairness[df_fairness['AB_WhiteMale']==1], agg_func=lambda df: df['correct_predict'].mean()))
print("A and B White Male - triplet accuracy: ", triplet_acc_fairness(df_fairness[df_fairness['AB_WhiteMale']==1]))
print("A and B White Male - f1 score:", f1_score(y_AB_WhiteMale, y_pred_AB_WhiteMale))
print("\n")
print("A and B Non White Male - mean accuracy: ", df_fairness[df_fairness['AB_NoWhiteMale']==1]['correct_predict'].mean())
print("A and B Non White Male - bootstrapping mean accuracy: ", bootstrap(df_fairness[df_fairness['AB_NoWhiteMale']==1], agg_func=lambda df: df['correct_predict'].mean()))
print("A and B Non White Male - triplet accuracy: ", triplet_acc_fairness(df_fairness[df_fairness['AB_NoWhiteMale']==1]))
print("A and B Non White Male - f1 score:", f1_score(y_AB_NoWhiteMale, y_pred_AB_NoWhiteMale))
print("\n")
print("A or B Non White Male - mean accuracy: ", df_fairness[df_fairness['AB_WhiteMale']==0]['correct_predict'].mean())
print("A or B Non White Male - bootstrapping mean accuracy: ", bootstrap(df_fairness[df_fairness['AB_WhiteMale']==0], agg_func=lambda df: df['correct_predict'].mean()))
print("A or B Non White Male - triplet accuracy: ", triplet_acc_fairness(df_fairness[df_fairness['AB_WhiteMale']==0]))
print("A or B Non White Male - f1 score:", f1_score(y_AoB_NoWhiteMale, y_pred_AoB_NoWhiteMale))

In [ ]:
print("A and B White Male - Confusion Matrix")
print(confusion_matrix(y_AB_WhiteMale, y_pred_AB_WhiteMale))
print("\n")
print('A and B White Male - TPR: ', TPR_AB_WhiteMale)
print('A and B White Male - FPR: ', FPR_AB_WhiteMale)
print('A and B White Male - TNR: ', TNR_AB_WhiteMale)
print('A and B White Male - FNR: ', FNR_AB_WhiteMale)

In [ ]:
print("A and B Non White Male - Confusion Matrix")
print(confusion_matrix(y_AB_NoWhiteMale, y_pred_AB_NoWhiteMale))
print("\n")
print('A and B Non White Male - TPR: ', TPR_AB_NoWhiteMale)
print('A and B Non White Male - FPR: ', FPR_AB_NoWhiteMale)
print('A and B Non White Male - TNR: ', TNR_AB_NoWhiteMale)
print('A and B Non White Male - FNR: ', FNR_AB_NoWhiteMale)

In [ ]:
print("A or B Non White Male - Confusion Matrix")
print(confusion_matrix(y_AoB_NoWhiteMale, y_pred_AoB_NoWhiteMale))
print("\n")
print('A or B Non White Male - TPR: ', TPR_AoB_NoWhiteMale)
print('A or B Non White Male - FPR: ', FPR_AoB_NoWhiteMale)
print('A or B Non White Male - TNR: ', TNR_AoB_NoWhiteMale)
print('A or B Non White Male - FNR: ', FNR_AoB_NoWhiteMale)

#### Male vs Non Male and White vs Non White

In [ ]:
pos_dist = df_fairness[(df_fairness.y_true==1) & (df_fairness['A_Male']==1)&(df_fairness['B_Male']==1)]['Distance']
neg_dist = df_fairness[(df_fairness.y_true==0) & (df_fairness['A_Male']==1)&(df_fairness['B_Male']==1)]['Distance']
print_hist_dist_zoom(pos_dist, neg_dist, zoom=5.)

In [ ]:
pos_dist = df_fairness[(df_fairness.y_true==1) & (df_fairness['A_Male']==0)&(df_fairness['B_Male']==0)]['Distance']
neg_dist = df_fairness[(df_fairness.y_true==0) & (df_fairness['A_Male']==0)&(df_fairness['B_Male']==0)]['Distance']
print_hist_dist_zoom(pos_dist, neg_dist, zoom=5.)

In [ ]:
pos_dist = df_fairness[(df_fairness.y_true==1) & (df_fairness['A_White']==1)&(df_fairness['B_White']==1)]['Distance']
neg_dist = df_fairness[(df_fairness.y_true==0) & (df_fairness['A_White']==1)&(df_fairness['B_White']==1)]['Distance']
print_hist_dist_zoom(pos_dist, neg_dist, zoom=5.)

In [ ]:
pos_dist = df_fairness[(df_fairness.y_true==1) & (df_fairness['A_White']==0)&(df_fairness['B_White']==0)]['Distance']
neg_dist = df_fairness[(df_fairness.y_true==0) & (df_fairness['A_White']==0)&(df_fairness['B_White']==0)]['Distance']
print_hist_dist_zoom(pos_dist, neg_dist, zoom=5.)

In [ ]:
y_AB_Male = np.array(df_fairness[(df_fairness['A_Male']==1)&(df_fairness['B_Male']==1)].y_true)
y_pred_AB_Male = np.array(df_fairness[(df_fairness['A_Male']==1)&(df_fairness['B_Male']==1)].y_pred)

tn_AB_Male,fp_AB_Male,fn_AB_Male,tp_AB_Male = confusion_matrix(y_AB_Male, y_pred_AB_Male).ravel()

TPR_AB_Male = tp_AB_Male/(tp_AB_Male+fp_AB_Male)
FPR_AB_Male = fp_AB_Male/(tp_AB_Male+fp_AB_Male)
TNR_AB_Male = tn_AB_Male/(tn_AB_Male+fn_AB_Male)
FNR_AB_Male = fn_AB_Male/(tn_AB_Male+fn_AB_Male)

y_AB_NoMale = np.array(df_fairness[(df_fairness['A_Male']==0)&(df_fairness['B_Male']==0)].y_true)
y_pred_AB_NoMale = np.array(df_fairness[(df_fairness['A_Male']==0)&(df_fairness['B_Male']==0)].y_pred)

tn_AB_NoMale,fp_AB_NoMale,fn_AB_NoMale,tp_AB_NoMale = confusion_matrix(y_AB_NoMale, y_pred_AB_NoMale).ravel()

TPR_AB_NoMale = tp_AB_NoMale/(tp_AB_NoMale+fp_AB_NoMale)
FPR_AB_NoMale = fp_AB_NoMale/(tp_AB_NoMale+fp_AB_NoMale)
TNR_AB_NoMale = tn_AB_NoMale/(tn_AB_NoMale+fn_AB_NoMale)
FNR_AB_NoMale = fn_AB_NoMale/(tn_AB_NoMale+fn_AB_NoMale)

In [ ]:
print("A and B Male - mean accuracy: ", df_fairness[(df_fairness['A_Male']==1)&(df_fairness['B_Male']==1)]['correct_predict'].mean())
print("A and B Male - bootstrapping mean accuracy: ", bootstrap(df_fairness[(df_fairness['A_Male']==1)&(df_fairness['B_Male']==1)], agg_func=lambda df: df['correct_predict'].mean()))
print("A and B Male - triplet accuracy: ", triplet_acc_fairness(df_fairness[(df_fairness['A_Male']==1)&(df_fairness['B_Male']==1)]))
print("A and B Male - f1 score:", f1_score(y_AB_Male, y_pred_AB_Male))
print("A and B Male - Confusion Matrix")
print(confusion_matrix(y_AB_Male, y_pred_AB_Male))
print('A and B Male - TPR: ', TPR_AB_Male)
print('A and B Male - FPR: ', FPR_AB_Male)
print('A and B Male - TNR: ', TNR_AB_Male)
print('A and B Male - FNR: ', FNR_AB_Male)
print("\n")
print("A and B Non Male - mean accuracy: ", df_fairness[(df_fairness['A_Male']==0)&(df_fairness['B_Male']==0)]['correct_predict'].mean())
print("A and B Non Male - bootstrapping mean accuracy: ", bootstrap(df_fairness[(df_fairness['A_Male']==0)&(df_fairness['B_Male']==0)], agg_func=lambda df: df['correct_predict'].mean()))
print("A and B Non Male - triplet accuracy: ", triplet_acc_fairness(df_fairness[(df_fairness['A_Male']==0)&(df_fairness['B_Male']==0)]))
print("A and B Non Male - f1 score:", f1_score(y_AB_NoMale, y_pred_AB_NoMale))
print("A and B Non Male - Confusion Matrix")
print(confusion_matrix(y_AB_NoMale, y_pred_AB_NoMale))
print('A and B Non Male - TPR: ', TPR_AB_NoMale)
print('A and B Non Male - FPR: ', FPR_AB_NoMale)
print('A and B Non Male - TNR: ', TNR_AB_NoMale)
print('A and B Non Male - FNR: ', FNR_AB_NoMale)

In [ ]:
y_AB_White = np.array(df_fairness[(df_fairness['A_White']==1)&(df_fairness['B_White']==1)].y_true)
y_pred_AB_White = np.array(df_fairness[(df_fairness['A_White']==1)&(df_fairness['B_White']==1)].y_pred)

tn_AB_White,fp_AB_White,fn_AB_White,tp_AB_White = confusion_matrix(y_AB_White, y_pred_AB_White).ravel()

TPR_AB_White = tp_AB_White/(tp_AB_White+fp_AB_White)
FPR_AB_White = fp_AB_White/(tp_AB_White+fp_AB_White)
TNR_AB_White = tn_AB_White/(tn_AB_White+fn_AB_White)
FNR_AB_White = fn_AB_White/(tn_AB_White+fn_AB_White)

y_AB_NoWhite = np.array(df_fairness[(df_fairness['A_White']==0)&(df_fairness['B_White']==0)].y_true)
y_pred_AB_NoWhite = np.array(df_fairness[(df_fairness['A_White']==0)&(df_fairness['B_White']==0)].y_pred)

tn_AB_NoWhite,fp_AB_NoWhite,fn_AB_NoWhite,tp_AB_NoWhite = confusion_matrix(y_AB_NoWhite, y_pred_AB_NoWhite).ravel()

TPR_AB_NoWhite = tp_AB_NoWhite/(tp_AB_NoWhite+fp_AB_NoWhite)
FPR_AB_NoWhite = fp_AB_NoWhite/(tp_AB_NoWhite+fp_AB_NoWhite)
TNR_AB_NoWhite = tn_AB_NoWhite/(tn_AB_NoWhite+fn_AB_NoWhite)
FNR_AB_NoWhite = fn_AB_NoWhite/(tn_AB_NoWhite+fn_AB_NoWhite)

In [ ]:
print("A and B White - mean accuracy: ", df_fairness[(df_fairness['A_White']==1)&(df_fairness['B_White']==1)]['correct_predict'].mean())
print("A and B White - bootstrapping mean accuracy: ", bootstrap(df_fairness[(df_fairness['A_White']==1)&(df_fairness['B_White']==1)], agg_func=lambda df: df['correct_predict'].mean()))
print("A and B White - triplet accuracy: ", triplet_acc_fairness(df_fairness[(df_fairness['A_White']==1)&(df_fairness['B_White']==1)]))
print("A and B White - f1 score:", f1_score(y_AB_White, y_pred_AB_White))
print("A and B White - Confusion Matrix")
print(confusion_matrix(y_AB_White, y_pred_AB_White))
print('A and B White - TPR: ', TPR_AB_White)
print('A and B White - FPR: ', FPR_AB_White)
print('A and B White - TNR: ', TNR_AB_White)
print('A and B White - FNR: ', FNR_AB_White)
print("\n")
print("A and B Non White - mean accuracy: ", df_fairness[(df_fairness['A_White']==0)&(df_fairness['B_White']==0)]['correct_predict'].mean())
print("A and B Non White - bootstrapping mean accuracy: ", bootstrap(df_fairness[(df_fairness['A_White']==0)&(df_fairness['B_White']==0)], agg_func=lambda df: df['correct_predict'].mean()))
print("A and B Non White - triplet accuracy: ", triplet_acc_fairness(df_fairness[(df_fairness['A_White']==0)&(df_fairness['B_White']==0)]))
print("A and B Non White - f1 score:", f1_score(y_AB_NoWhite, y_pred_AB_NoWhite))
print("A and B Non White - Confusion Matrix")
print(confusion_matrix(y_AB_NoWhite, y_pred_AB_NoWhite))
print('A and B Non White - TPR: ', TPR_AB_NoWhite)
print('A and B Non White - FPR: ', FPR_AB_NoWhite)
print('A and B Non White - TNR: ', TNR_AB_NoWhite)
print('A and B Non White - FNR: ', FNR_AB_NoWhite)

### Other categories

#### Supposed races

In [ ]:
y_AB_Black = np.array(df_fairness[(df_fairness['A_Black']==1)&(df_fairness['B_Black']==1)].y_true)
y_pred_AB_Black = np.array(df_fairness[(df_fairness['A_Black']==1)&(df_fairness['B_Black']==1)].y_pred)

tn_AB_Black,fp_AB_Black,fn_AB_Black,tp_AB_Black = confusion_matrix(y_AB_Black, y_pred_AB_Black).ravel()

TPR_AB_Black = tp_AB_Black/(tp_AB_Black+fp_AB_Black)
FPR_AB_Black = fp_AB_Black/(tp_AB_Black+fp_AB_Black)
TNR_AB_Black = tn_AB_Black/(tn_AB_Black+fn_AB_Black)
FNR_AB_Black = fn_AB_Black/(tn_AB_Black+fn_AB_Black)

y_AB_Asian = np.array(df_fairness[(df_fairness['A_Asian']==1)&(df_fairness['B_Asian']==1)].y_true)
y_pred_AB_Asian = np.array(df_fairness[(df_fairness['A_Asian']==1)&(df_fairness['B_Asian']==1)].y_pred)

tn_AB_Asian,fp_AB_Asian,fn_AB_Asian,tp_AB_Asian = confusion_matrix(y_AB_Asian, y_pred_AB_Asian).ravel()

TPR_AB_Asian = tp_AB_Asian/(tp_AB_Asian+fp_AB_Asian)
FPR_AB_Asian = fp_AB_Asian/(tp_AB_Asian+fp_AB_Asian)
TNR_AB_Asian = tn_AB_Asian/(tn_AB_Asian+fn_AB_Asian)
FNR_AB_Asian = fn_AB_Asian/(tn_AB_Asian+fn_AB_Asian)

y_AB_Indian = np.array(df_fairness[(df_fairness['A_Indian']==1)&(df_fairness['B_Indian']==1)].y_true)
y_pred_AB_Indian = np.array(df_fairness[(df_fairness['A_Indian']==1)&(df_fairness['B_Indian']==1)].y_pred)

tn_AB_Indian,fp_AB_Indian,fn_AB_Indian,tp_AB_Indian = confusion_matrix(y_AB_Indian, y_pred_AB_Indian).ravel()

TPR_AB_Indian = tp_AB_Indian/(tp_AB_Indian+fp_AB_Indian)
FPR_AB_Indian = fp_AB_Indian/(tp_AB_Indian+fp_AB_Indian)
TNR_AB_Indian = tn_AB_Indian/(tn_AB_Indian+fn_AB_Indian)
FNR_AB_Indian = fn_AB_Indian/(tn_AB_Indian+fn_AB_Indian)

In [ ]:
print("A and B White - mean accuracy: ", df_fairness[(df_fairness['A_White']==1)&(df_fairness['B_White']==1)]['correct_predict'].mean())
print("A and B White - bootstrapping mean accuracy: ", bootstrap(df_fairness[(df_fairness['A_White']==1)&(df_fairness['B_White']==1)], agg_func=lambda df: df['correct_predict'].mean()))
print("A and B White - triplet accuracy: ", triplet_acc_fairness(df_fairness[(df_fairness['A_White']==1)&(df_fairness['B_White']==1)]))
print("A and B White - f1 score:", f1_score(y_AB_White, y_pred_AB_White))
print("A and B White - Confusion Matrix")
print(confusion_matrix(y_AB_White, y_pred_AB_White))
print('A and B White - TPR: ', TPR_AB_White)
print('A and B White - FPR: ', FPR_AB_White)
print('A and B White - TNR: ', TNR_AB_White)
print('A and B White - FNR: ', FNR_AB_White)
print("\n")
print("A and B Black - mean accuracy: ", df_fairness[(df_fairness['A_Black']==1)&(df_fairness['B_Black']==1)]['correct_predict'].mean())
print("A and B Black - bootstrapping mean accuracy: ", bootstrap(df_fairness[(df_fairness['A_Black']==1)&(df_fairness['B_Black']==1)], agg_func=lambda df: df['correct_predict'].mean()))
print("A and B Black - triplet accuracy: ", triplet_acc_fairness(df_fairness[(df_fairness['A_Black']==1)&(df_fairness['B_Black']==1)]))
print("A and B Black - f1 score:", f1_score(y_AB_Black, y_pred_AB_Black))
print("A and B Black - Confusion Matrix")
print(confusion_matrix(y_AB_Black, y_pred_AB_Black))
print('A and B Black - TPR: ', TPR_AB_Black)
print('A and B Black - FPR: ', FPR_AB_Black)
print('A and B Black - TNR: ', TNR_AB_Black)
print('A and B Black - FNR: ', FNR_AB_Black)
print("\n")
print("A and B Asian - mean accuracy: ", df_fairness[(df_fairness['A_Asian']==1)&(df_fairness['B_Asian']==1)]['correct_predict'].mean())
print("A and B Asian - bootstrapping mean accuracy: ", bootstrap(df_fairness[(df_fairness['A_Asian']==1)&(df_fairness['B_Asian']==1)], agg_func=lambda df: df['correct_predict'].mean()))
print("A and B Asian - triplet accuracy: ", triplet_acc_fairness(df_fairness[(df_fairness['A_Asian']==1)&(df_fairness['B_Asian']==1)]))
print("A and B Asian - f1 score:", f1_score(y_AB_Asian, y_pred_AB_Asian))
print("A and B Asian - Confusion Matrix")
print(confusion_matrix(y_AB_Asian, y_pred_AB_Asian))
print('A and B Asian - TPR: ', TPR_AB_Asian)
print('A and B Asian - FPR: ', FPR_AB_Asian)
print('A and B Asian - TNR: ', TNR_AB_Asian)
print('A and B Asian - FNR: ', FNR_AB_Asian)
print("\n")
print("A and B Indian - mean accuracy: ", df_fairness[(df_fairness['A_Indian']==1)&(df_fairness['B_Indian']==1)]['correct_predict'].mean())
print("A and B Indian - bootstrapping mean accuracy: ", bootstrap(df_fairness[(df_fairness['A_Indian']==1)&(df_fairness['B_Indian']==1)], agg_func=lambda df: df['correct_predict'].mean()))
print("A and B Indian - triplet accuracy: ", triplet_acc_fairness(df_fairness[(df_fairness['A_Indian']==1)&(df_fairness['B_Indian']==1)]))
print("A and B Indian - f1 score:", f1_score(y_AB_Indian, y_pred_AB_Indian))
print("A and B Indian - Confusion Matrix")
print(confusion_matrix(y_AB_Indian, y_pred_AB_Indian))
print('A and B Indian - TPR: ', TPR_AB_Indian)
print('A and B Indian - FPR: ', FPR_AB_Indian)
print('A and B Indian - TNR: ', TNR_AB_Indian)
print('A and B Indian - FNR: ', FNR_AB_Indian)

#### Youth vs Senior

In [ ]:
y_AB_Youth = np.array(df_fairness[(df_fairness['A_Youth']==1)&(df_fairness['B_Youth']==1)].y_true)
y_pred_AB_Youth = np.array(df_fairness[(df_fairness['A_Youth']==1)&(df_fairness['B_Youth']==1)].y_pred)

tn_AB_Youth,fp_AB_Youth,fn_AB_Youth,tp_AB_Youth = confusion_matrix(y_AB_Youth, y_pred_AB_Youth).ravel()

TPR_AB_Youth = tp_AB_Youth/(tp_AB_Youth+fp_AB_Youth)
FPR_AB_Youth = fp_AB_Youth/(tp_AB_Youth+fp_AB_Youth)
TNR_AB_Youth = tn_AB_Youth/(tn_AB_Youth+fn_AB_Youth)
FNR_AB_Youth = fn_AB_Youth/(tn_AB_Youth+fn_AB_Youth)

y_AB_Senior = np.array(df_fairness[(df_fairness['A_Senior']==1)&(df_fairness['B_Senior']==1)].y_true)
y_pred_AB_Senior = np.array(df_fairness[(df_fairness['A_Senior']==1)&(df_fairness['B_Senior']==1)].y_pred)

tn_AB_Senior,fp_AB_Senior,fn_AB_Senior,tp_AB_Senior = confusion_matrix(y_AB_Senior, y_pred_AB_Senior).ravel()

TPR_AB_Senior = tp_AB_Senior/(tp_AB_Senior+fp_AB_Senior)
FPR_AB_Senior = fp_AB_Senior/(tp_AB_Senior+fp_AB_Senior)
TNR_AB_Senior = tn_AB_Senior/(tn_AB_Senior+fn_AB_Senior)
FNR_AB_Senior = fn_AB_Senior/(tn_AB_Senior+fn_AB_Senior)

In [ ]:
print("A and B Youth - mean accuracy: ", df_fairness[(df_fairness['A_Youth']==1)&(df_fairness['B_Youth']==1)]['correct_predict'].mean())
print("A and B Youth - bootstrapping mean accuracy: ", bootstrap(df_fairness[(df_fairness['A_Youth']==1)&(df_fairness['B_Youth']==1)], agg_func=lambda df: df['correct_predict'].mean()))
print("A and B Youth - triplet accuracy: ", triplet_acc_fairness(df_fairness[(df_fairness['A_Youth']==1)&(df_fairness['B_Youth']==1)]))
print("A and B Youth - f1 score:", f1_score(y_AB_Youth, y_pred_AB_Youth))
print("A and B Youth - Confusion Matrix")
print(confusion_matrix(y_AB_Youth, y_pred_AB_Youth))
print('A and B Youth - TPR: ', TPR_AB_Youth)
print('A and B Youth - FPR: ', FPR_AB_Youth)
print('A and B Youth - TNR: ', TNR_AB_Youth)
print('A and B Youth - FNR: ', FNR_AB_Youth)
print("\n")
print("A and B Senior - mean accuracy: ", df_fairness[(df_fairness['A_Senior']==1)&(df_fairness['B_Senior']==1)]['correct_predict'].mean())
print("A and B Senior - bootstrapping mean accuracy: ", bootstrap(df_fairness[(df_fairness['A_Senior']==1)&(df_fairness['B_Senior']==1)], agg_func=lambda df: df['correct_predict'].mean()))
print("A and B Senior - triplet accuracy: ", triplet_acc_fairness(df_fairness[(df_fairness['A_Senior']==1)&(df_fairness['B_Senior']==1)]))
print("A and B Senior - f1 score:", f1_score(y_AB_Senior, y_pred_AB_Senior))
print("A and B Senior - Confusion Matrix")
print(confusion_matrix(y_AB_Senior, y_pred_AB_Senior))
print('A and B Senior - TPR: ', TPR_AB_Senior)
print('A and B Senior - FPR: ', FPR_AB_Senior)
print('A and B Senior - TNR: ', TNR_AB_Senior)
print('A and B Senior - FNR: ', FNR_AB_Senior)

#### Accessories

In [ ]:
y_AB_Sunglasses = np.array(df_fairness[(df_fairness['A_Sunglasses']==1)&(df_fairness['B_Sunglasses']==1)].y_true)
y_pred_AB_Sunglasses = np.array(df_fairness[(df_fairness['A_Sunglasses']==1)&(df_fairness['B_Sunglasses']==1)].y_pred)

tn_AB_Sunglasses,fp_AB_Sunglasses,fn_AB_Sunglasses,tp_AB_Sunglasses = confusion_matrix(y_AB_Sunglasses, y_pred_AB_Sunglasses).ravel()

TPR_AB_Sunglasses = tp_AB_Sunglasses/(tp_AB_Sunglasses+fp_AB_Sunglasses)
FPR_AB_Sunglasses = fp_AB_Sunglasses/(tp_AB_Sunglasses+fp_AB_Sunglasses)
TNR_AB_Sunglasses = tn_AB_Sunglasses/(tn_AB_Sunglasses+fn_AB_Sunglasses)
FNR_AB_Sunglasses = fn_AB_Sunglasses/(tn_AB_Sunglasses+fn_AB_Sunglasses)

y_AB_NoSunglasses = np.array(df_fairness[(df_fairness['A_Sunglasses']==0)&(df_fairness['B_Sunglasses']==0)].y_true)
y_pred_AB_NoSunglasses = np.array(df_fairness[(df_fairness['A_Sunglasses']==0)&(df_fairness['B_Sunglasses']==0)].y_pred)

tn_AB_NoSunglasses,fp_AB_NoSunglasses,fn_AB_NoSunglasses,tp_AB_NoSunglasses = confusion_matrix(y_AB_NoSunglasses, y_pred_AB_NoSunglasses).ravel()

TPR_AB_NoSunglasses = tp_AB_NoSunglasses/(tp_AB_NoSunglasses+fp_AB_NoSunglasses)
FPR_AB_NoSunglasses = fp_AB_NoSunglasses/(tp_AB_NoSunglasses+fp_AB_NoSunglasses)
TNR_AB_NoSunglasses = tn_AB_NoSunglasses/(tn_AB_NoSunglasses+fn_AB_NoSunglasses)
FNR_AB_NoSunglasses = fn_AB_NoSunglasses/(tn_AB_NoSunglasses+fn_AB_NoSunglasses)

In [ ]:
print("A and B Sunglasses - mean accuracy: ", df_fairness[(df_fairness['A_Sunglasses']==1)&(df_fairness['B_Sunglasses']==1)]['correct_predict'].mean())
print("A and B Sunglasses - bootstrapping mean accuracy: ", bootstrap(df_fairness[(df_fairness['A_Sunglasses']==1)&(df_fairness['B_Sunglasses']==1)], agg_func=lambda df: df['correct_predict'].mean(), num_bootstraps=10000))
print("A and B Sunglasses - triplet accuracy: ", triplet_acc_fairness(df_fairness[(df_fairness['A_Sunglasses']==1)&(df_fairness['B_Sunglasses']==1)]))
print("A and B Sunglasses - f1 score:", f1_score(y_AB_Sunglasses, y_pred_AB_Sunglasses))
print("A and B Sunglasses - Confusion Matrix")
print(confusion_matrix(y_AB_Sunglasses, y_pred_AB_Sunglasses))
print('A and B Sunglasses - TPR: ', TPR_AB_Sunglasses)
print('A and B Sunglasses - FPR: ', FPR_AB_Sunglasses)
print('A and B Sunglasses - TNR: ', TNR_AB_Sunglasses)
print('A and B Sunglasses - FNR: ', FNR_AB_Sunglasses)
print("\n")
print("A and B Non Sunglasses - mean accuracy: ", df_fairness[(df_fairness['A_Sunglasses']==0)&(df_fairness['B_Sunglasses']==0)]['correct_predict'].mean())
print("A and B Non Sunglasses - bootstrapping mean accuracy: ", bootstrap(df_fairness[(df_fairness['A_Sunglasses']==0)&(df_fairness['B_Sunglasses']==0)], agg_func=lambda df: df['correct_predict'].mean(), num_bootstraps=10000))
print("A and B Non Sunglasses - triplet accuracy: ", triplet_acc_fairness(df_fairness[(df_fairness['A_Sunglasses']==0)&(df_fairness['B_Sunglasses']==0)]))
print("A and B Non Sunglasses - f1 score:", f1_score(y_AB_NoSunglasses, y_pred_AB_NoSunglasses))
print("A and B Non Sunglasses - Confusion Matrix")
print(confusion_matrix(y_AB_NoSunglasses, y_pred_AB_NoSunglasses))
print('A and B Non Sunglasses - TPR: ', TPR_AB_NoSunglasses)
print('A and B Non Sunglasses - FPR: ', FPR_AB_NoSunglasses)
print('A and B Non Sunglasses - TNR: ', TNR_AB_NoSunglasses)
print('A and B Non Sunglasses - FNR: ', FNR_AB_NoSunglasses)

#### More bootstrapping

In [ ]:
b,a,c = np.round_(bootstrap(df_fairness[df_fairness['AB_WhiteMale']==1], \
    agg_func=lambda df: df['correct_predict'].mean(), num_bootstraps=10000, percentiles=[5,50,95]), 3)

print(f"A and B White Male - accuracy: {a} (90% CI: {b}-{c})")

b,a,c = np.round_(bootstrap_by_pairs(df_fairness[df_fairness['AB_WhiteMale']==1], \
    agg_func=lambda df: triplet_acc_fairness(df), num_bootstraps=100, percentiles=[5,50,95]), 3)

print(f"A and B White Male - triplet accuracy: {a} (90% CI: {b}-{c})")

b,a,c = np.round_(bootstrap(df_fairness[df_fairness['AB_WhiteMale']==1], \
    agg_func=lambda df: df[(df['y_pred']==1)&(df['y_true']==0)]['id_A'].count()/df[df['y_pred']==1]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3)

print(f"A and B White Male - FPR: {a} (90% CI: {b}-{c})")

b,a,c = np.round_(bootstrap(df_fairness[df_fairness['AB_WhiteMale']==1], \
    agg_func=lambda df: df[(df['y_pred']==0)&(df['y_true']==1)]['id_A'].count()/df[df['y_pred']==0]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3)

print(f"A and B White Male - FNR: {a} (90% CI: {b}-{c})")

print("\n")

b,a,c = np.round_(bootstrap(df_fairness[df_fairness['AB_NoWhiteMale']==1], \
    agg_func=lambda df: df['correct_predict'].mean(), num_bootstraps=10000, percentiles=[5,50,95]),3)

print(f"A and B Non White Male - accuracy: {a} (90% CI: {b}-{c})")

b,a,c = np.round_(bootstrap_by_pairs(df_fairness[df_fairness['AB_NoWhiteMale']==1], \
    agg_func=lambda df: triplet_acc_fairness(df), num_bootstraps=100, percentiles=[5,50,95]), 3)

print(f"A and B Non White Male - triplet accuracy: {a} (90% CI: {b}-{c})")

b,a,c = np.round_(bootstrap(df_fairness[df_fairness['AB_NoWhiteMale']==1], \
    agg_func=lambda df: df[(df['y_pred']==1)&(df['y_true']==0)]['id_A'].count()/df[df['y_pred']==1]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3)
 
print(f"A and B Non White Male - FPR: {a} (90% CI: {b}-{c})")

b,a,c = np.round_(bootstrap(df_fairness[df_fairness['AB_NoWhiteMale']==1], \
    agg_func=lambda df: df[(df['y_pred']==0)&(df['y_true']==1)]['id_A'].count()/df[df['y_pred']==0]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3)

print(f"A and B Non White Male - FNR: {a} (90% CI: {b}-{c})")

In [ ]:
b,a,c = np.round_(bootstrap(df_fairness[(df_fairness['A_Male']==1)&(df_fairness['B_Male']==1)], \
    agg_func=lambda df: df['correct_predict'].mean(), num_bootstraps=10000, percentiles=[5,50,95]), 3)

print(f"A and B Male - accuracy: {a} (90% CI: {b}-{c})")

b,a,c = np.round_(bootstrap_by_pairs(df_fairness[(df_fairness['A_Male']==1)&(df_fairness['B_Male']==1)], \
    agg_func=lambda df: triplet_acc_fairness(df), num_bootstraps=100, percentiles=[5,50,95]), 3)

print(f"A and B Male - triplet accuracy: {a} (90% CI: {b}-{c})")

b,a,c = np.round_(bootstrap(df_fairness[(df_fairness['A_Male']==1)&(df_fairness['B_Male']==1)], \
    agg_func=lambda df: df[(df['y_pred']==1)&(df['y_true']==0)]['id_A'].count()/df[df['y_pred']==1]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3)

print(f"A and B Male - FPR: {a} (90% CI: {b}-{c})")

b,a,c = np.round_(bootstrap(df_fairness[(df_fairness['A_Male']==1)&(df_fairness['B_Male']==1)], \
    agg_func=lambda df: df[(df['y_pred']==0)&(df['y_true']==1)]['id_A'].count()/df[df['y_pred']==0]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3)

print(f"A and B Male - FNR: {a} (90% CI: {b}-{c})")

print("\n")

b,a,c = np.round_(bootstrap(df_fairness[(df_fairness['A_Male']==0)&(df_fairness['B_Male']==0)], \
    agg_func=lambda df: df['correct_predict'].mean(), num_bootstraps=10000, percentiles=[5,50,95]),3)

print(f"A and B Non Male - accuracy: {a} (90% CI: {b}-{c})")

b,a,c = np.round_(bootstrap_by_pairs(df_fairness[(df_fairness['A_Male']==0)&(df_fairness['B_Male']==0)], \
    agg_func=lambda df: triplet_acc_fairness(df), num_bootstraps=100, percentiles=[5,50,95]), 3)

print(f"A and B Non Male - triplet accuracy: {a} (90% CI: {b}-{c})")

b,a,c = np.round_(bootstrap(df_fairness[(df_fairness['A_Male']==0)&(df_fairness['B_Male']==0)], \
    agg_func=lambda df: df[(df['y_pred']==1)&(df['y_true']==0)]['id_A'].count()/df[df['y_pred']==1]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3)

print(f"A and B Non Male - FPR: {a} (90% CI: {b}-{c})")

b,a,c = np.round_(bootstrap(df_fairness[(df_fairness['A_Male']==0)&(df_fairness['B_Male']==0)], \
    agg_func=lambda df: df[(df['y_pred']==0)&(df['y_true']==1)]['id_A'].count()/df[df['y_pred']==0]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3)

print(f"A and B Non Male - FNR: {a} (90% CI: {b}-{c})")

In [ ]:
b,a,c = np.round_(bootstrap(df_fairness[(df_fairness['A_White']==1)&(df_fairness['B_White']==1)], \
    agg_func=lambda df: df['correct_predict'].mean(), num_bootstraps=10000, percentiles=[5,50,95]), 3)

print(f"A and B White - accuracy: {a} (90% CI: {b}-{c})")

b,a,c = np.round_(bootstrap_by_pairs(df_fairness[(df_fairness['A_White']==1)&(df_fairness['B_White']==1)], \
    agg_func=lambda df: triplet_acc_fairness(df), num_bootstraps=100, percentiles=[5,50,95]), 3)

print(f"A and B White - triplet accuracy: {a} (90% CI: {b}-{c})")

b,a,c = np.round_(bootstrap(df_fairness[(df_fairness['A_White']==1)&(df_fairness['B_White']==1)], \
    agg_func=lambda df: df[(df['y_pred']==1)&(df['y_true']==0)]['id_A'].count()/df[df['y_pred']==1]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3)

print(f"A and B White - FPR: {a} (90% CI: {b}-{c})")

b,a,c = np.round_(bootstrap(df_fairness[(df_fairness['A_White']==1)&(df_fairness['B_White']==1)], \
    agg_func=lambda df: df[(df['y_pred']==0)&(df['y_true']==1)]['id_A'].count()/df[df['y_pred']==0]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3)

print(f"A and B White - FNR: {a} (90% CI: {b}-{c})")

print("\n")

b,a,c = np.round_(bootstrap(df_fairness[(df_fairness['A_White']==0)&(df_fairness['B_White']==0)], \
    agg_func=lambda df: df['correct_predict'].mean(), num_bootstraps=10000, percentiles=[5,50,95]),3)

print(f"A and B Non White - accuracy: {a} (90% CI: {b}-{c})")

b,a,c = np.round_(bootstrap_by_pairs(df_fairness[(df_fairness['A_White']==0)&(df_fairness['B_White']==0)], \
    agg_func=lambda df: triplet_acc_fairness(df), num_bootstraps=100, percentiles=[5,50,95]), 3)

print(f"A and B Non White - triplet accuracy: {a} (90% CI: {b}-{c})")

b,a,c = np.round_(bootstrap(df_fairness[(df_fairness['A_White']==0)&(df_fairness['B_White']==0)], \
    agg_func=lambda df: df[(df['y_pred']==1)&(df['y_true']==0)]['id_A'].count()/df[df['y_pred']==1]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3)

print(f"A and B Non White - FPR: {a} (90% CI: {b}-{c})")

b,a,c = np.round_(bootstrap(df_fairness[(df_fairness['A_White']==0)&(df_fairness['B_White']==0)], \
    agg_func=lambda df: df[(df['y_pred']==0)&(df['y_true']==1)]['id_A'].count()/df[df['y_pred']==0]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3)

print(f"A and B Non White - FNR: {a} (90% CI: {b}-{c})")

In [ ]:
print("A and B White - mean accuracy: ", np.round_(bootstrap(df_fairness[(df_fairness['A_White']==1)&(df_fairness['B_White']==1)], \
    agg_func=lambda df: df['correct_predict'].mean(), num_bootstraps=10000, percentiles=[5,50,95]), 3))
print("A and B White - triplet accuracy: ", np.round_(bootstrap_by_pairs(df_fairness[(df_fairness['A_White']==1)&(df_fairness['B_White']==1)], \
    agg_func=lambda df: triplet_acc_fairness(df), num_bootstraps=100, percentiles=[5,50,95]), 3))
print("A and B White - FPR: ", np.round_(bootstrap(df_fairness[(df_fairness['A_White']==1)&(df_fairness['B_White']==1)], \
    agg_func=lambda df: df[(df['y_pred']==1)&(df['y_true']==0)]['id_A'].count()/df[df['y_pred']==1]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3))
print("A and B White - FNR: ", np.round_(bootstrap(df_fairness[(df_fairness['A_White']==1)&(df_fairness['B_White']==1)], \
    agg_func=lambda df: df[(df['y_pred']==0)&(df['y_true']==1)]['id_A'].count()/df[df['y_pred']==0]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3))
print("\n")
print("A and B Black - mean accuracy: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Black']==1)&(df_fairness['B_Black']==1)], \
    agg_func=lambda df: df['correct_predict'].mean(), num_bootstraps=10000, percentiles=[5,50,95]),3))
print("A and B Black - triplet accuracy: ", np.round_(bootstrap_by_pairs(df_fairness[(df_fairness['A_Black']==1)&(df_fairness['B_Black']==1)], \
    agg_func=lambda df: triplet_acc_fairness(df), num_bootstraps=100, percentiles=[5,50,95]), 3))
print("A and B Black - FPR: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Black']==1)&(df_fairness['B_Black']==1)], \
    agg_func=lambda df: df[(df['y_pred']==1)&(df['y_true']==0)]['id_A'].count()/df[df['y_pred']==1]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3))
print("A and B Black - FNR: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Black']==1)&(df_fairness['B_Black']==1)], \
    agg_func=lambda df: df[(df['y_pred']==0)&(df['y_true']==1)]['id_A'].count()/df[df['y_pred']==0]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3))
print("\n")
print("A and B Asian - mean accuracy: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Asian']==1)&(df_fairness['B_Asian']==1)], \
    agg_func=lambda df: df['correct_predict'].mean(), num_bootstraps=10000, percentiles=[5,50,95]), 3))
print("A and B Asian - triplet accuracy: ", np.round_(bootstrap_by_pairs(df_fairness[(df_fairness['A_Asian']==1)&(df_fairness['B_Asian']==1)], \
    agg_func=lambda df: triplet_acc_fairness(df), num_bootstraps=100, percentiles=[5,50,95]), 3))
print("A and B Asian - FPR: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Asian']==1)&(df_fairness['B_Asian']==1)], \
    agg_func=lambda df: df[(df['y_pred']==1)&(df['y_true']==0)]['id_A'].count()/df[df['y_pred']==1]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3))
print("A and B Asian - FNR: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Asian']==1)&(df_fairness['B_Asian']==1)], \
    agg_func=lambda df: df[(df['y_pred']==0)&(df['y_true']==1)]['id_A'].count()/df[df['y_pred']==0]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3))
print("\n")
print("A and B Indian - mean accuracy: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Indian']==1)&(df_fairness['B_Indian']==1)], \
    agg_func=lambda df: df['correct_predict'].mean(), num_bootstraps=10000, percentiles=[5,50,95]),3))
print("A and B Indian - triplet accuracy: ", np.round_(bootstrap_by_pairs(df_fairness[(df_fairness['A_Indian']==1)&(df_fairness['B_Indian']==1)], \
    agg_func=lambda df: triplet_acc_fairness(df), num_bootstraps=100, percentiles=[5,50,95]), 3))
print("A and B Indian - FPR: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Indian']==1)&(df_fairness['B_Indian']==1)], \
    agg_func=lambda df: df[(df['y_pred']==1)&(df['y_true']==0)]['id_A'].count()/df[df['y_pred']==1]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3))
print("A and B Indian - FNR: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Indian']==1)&(df_fairness['B_Indian']==1)], \
    agg_func=lambda df: df[(df['y_pred']==0)&(df['y_true']==1)]['id_A'].count()/df[df['y_pred']==0]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3))

In [ ]:
print("A and B Youth - mean accuracy: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Youth']==1)&(df_fairness['B_Youth']==1)], \
    agg_func=lambda df: df['correct_predict'].mean(), num_bootstraps=10000, percentiles=[5,50,95]), 3))
print("A and B Youth - triplet accuracy: ", np.round_(bootstrap_by_pairs(df_fairness[(df_fairness['A_Youth']==1)&(df_fairness['B_Youth']==1)], \
    agg_func=lambda df: triplet_acc_fairness(df), num_bootstraps=100, percentiles=[5,50,95]), 3))
print("A and B Youth - FPR: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Youth']==1)&(df_fairness['B_Youth']==1)], \
    agg_func=lambda df: df[(df['y_pred']==1)&(df['y_true']==0)]['id_A'].count()/df[df['y_pred']==1]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3))
print("A and B Youth - FNR: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Youth']==1)&(df_fairness['B_Youth']==1)], \
    agg_func=lambda df: df[(df['y_pred']==0)&(df['y_true']==1)]['id_A'].count()/df[df['y_pred']==0]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3))
print("\n")
print("A and B Senior - mean accuracy: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Senior']==1)&(df_fairness['B_Senior']==1)], \
    agg_func=lambda df: df['correct_predict'].mean(), num_bootstraps=10000, percentiles=[5,50,95]),3))
print("A and B Senior - triplet accuracy: ", np.round_(bootstrap_by_pairs(df_fairness[(df_fairness['A_Senior']==1)&(df_fairness['B_Senior']==1)], \
    agg_func=lambda df: triplet_acc_fairness(df), num_bootstraps=100, percentiles=[5,50,95]), 3))
print("A and B Senior - FPR: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Senior']==1)&(df_fairness['B_Senior']==1)], \
    agg_func=lambda df: df[(df['y_pred']==1)&(df['y_true']==0)]['id_A'].count()/df[df['y_pred']==1]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3))
print("A and B Senior - FNR: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Senior']==1)&(df_fairness['B_Senior']==1)], \
    agg_func=lambda df: df[(df['y_pred']==0)&(df['y_true']==1)]['id_A'].count()/df[df['y_pred']==0]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3))

In [ ]:
print("A and B Sunglasses - mean accuracy: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Sunglasses']==1)&(df_fairness['B_Sunglasses']==1)], \
    agg_func=lambda df: df['correct_predict'].mean(), num_bootstraps=10000, percentiles=[5,50,95]), 3))
print("A and B Sunglasses - triplet accuracy: ", np.round_(bootstrap_by_pairs(df_fairness[(df_fairness['A_Sunglasses']==1)&(df_fairness['B_Sunglasses']==1)], \
    agg_func=lambda df: triplet_acc_fairness(df), num_bootstraps=100, percentiles=[5,50,95]), 3))
print("A and B Sunglasses - FPR: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Sunglasses']==1)&(df_fairness['B_Sunglasses']==1)], \
    agg_func=lambda df: df[(df['y_pred']==1)&(df['y_true']==0)]['id_A'].count()/df[df['y_pred']==1]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3))
print("A and B Sunglasses - FNR: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Sunglasses']==1)&(df_fairness['B_Sunglasses']==1)], \
    agg_func=lambda df: df[(df['y_pred']==0)&(df['y_true']==1)]['id_A'].count()/df[df['y_pred']==0]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3))
print("\n")
print("A and B Non Sunglasses - mean accuracy: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Sunglasses']==0)&(df_fairness['B_Sunglasses']==0)], \
    agg_func=lambda df: df['correct_predict'].mean(), num_bootstraps=10000, percentiles=[5,50,95]),3))
print("A and B Non Sunglasses - triplet accuracy: ", np.round_(bootstrap_by_pairs(df_fairness[(df_fairness['A_Sunglasses']==0)&(df_fairness['B_Sunglasses']==0)], \
    agg_func=lambda df: triplet_acc_fairness(df), num_bootstraps=100, percentiles=[5,50,95]), 3))
print("A and B Non Sunglasses - FPR: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Sunglasses']==0)&(df_fairness['B_Sunglasses']==0)], \
    agg_func=lambda df: df[(df['y_pred']==1)&(df['y_true']==0)]['id_A'].count()/df[df['y_pred']==1]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3))
print("A and B Non Sunglasses - FNR: ", np.round_(bootstrap(df_fairness[(df_fairness['A_Sunglasses']==0)&(df_fairness['B_Sunglasses']==0)], \
    agg_func=lambda df: df[(df['y_pred']==0)&(df['y_true']==1)]['id_A'].count()/df[df['y_pred']==0]['id_A'].count(), \
    num_bootstraps=10000, percentiles=[5,50,95]),3))